# GraphMAE
#intro 以mask rate=0.5进行实验，num_mask_nodes = int(0.5 * num_nodes) 保持可见的节点数量一致

######## distribution masking 
num_mask_nodes = int(mask_rate * num_nodes)
idx_val   = g.ndata['val_mask'].nonzero().squeeze()
idx_test  = g.ndata['test_mask'].nonzero().squeeze()
idx_train = g.ndata['train_mask'].nonzero().squeeze()

idx_mask = torch.cat((idx_test, idx_val))
idx_keep = idx_train  # 可以设计实验不断的减小有效信息的可见范围

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()
num_mask_nodes = len(mask_nodes)


#Cora 
test->train: torch.cat((idx_test, idx_val)) -> idx_train   30.18%
train->test: idx_train -> torch.cat((idx_test, idx_val))   62.58%

#Citeseer
test->train: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train   49.35%
train->test: idx_train -> torch.cat((idx_test, idx_val))                    63.98%

#CoraML
test->train: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train   27.80%
train->test: idx_train -> torch.cat((idx_test, idx_val))[:num_mask_nodes]   57.83%


Strategy                             & Attacked-Cora       &Attacked-Citeseer   & Attacked-CoraML
test->train                          & 30.18%              &49.35%              &27.80%   \\
random                               & 48.74%              &62.50%              &41.50%   \\
train->test                          & 62.58%              &63.98%              &57.83%   \\

#######################################################
#######################################################
#######################################################
#######################################################
#######################################################

# 不同的训练分布，发现只要虽然train的比例高达90%，但只要训练的epoch足够多，错误的引导一定会误导encoder
# 这里利用3000个epoch做实验 
# 比较对象为 
# train/val/test 比例为0.9/0.05/0.05 


#Cora 0.9/0.05/0.05的情况主要攻击test，但train的比例太高了，所以攻击有限
test->train: idx_test  -> torch.cat((idx_train, idx_val))
train->test: torch.cat((idx_train, idx_val)) -> idx_test 



# 在2000个epoch之后才逐渐开始越来越差  train/val/test 0.9/0.05/0.05
Strategy                             & Attacked-Cora      
test->train                          & 74.40           \\
random                               & 73.60           \\

train->test                          & 55.20%          \\ # 除了seed 1，其余seed都会衰减
# 补充一个同比例的实验，利用剩下的重建0.05的train，发现并不会误导encoder的学习
train->test                          & 74.40%          \\


# GraphMAE2
#intro 以mask rate=0.5进行实验，num_mask_nodes = int(0.5 * num_nodes) 保持可见的节点数量一致

######## distribution masking (add ssh)
num_mask_nodes = int(mask_rate * num_nodes)
idx_val   = g.ndata['val_mask'].nonzero().squeeze()
idx_test  = g.ndata['test_mask'].nonzero().squeeze()
idx_train = g.ndata['train_mask'].nonzero().squeeze()
# 确定keep和mask的分布范围
idx_keep_dis = torch.cat((idx_test, idx_val))
idx_mask_dis = idx_train  # 可以设计实验不断的减小有效信息的可见范围
# 在各自的分布范围内进行随机选择
perm_keep = torch.randperm(len(idx_keep_dis), device=x.device)
perm_mask = torch.randperm(len(idx_mask_dis), device=x.device)
# 挑选keep和mask的节点 #仅修改这里perm_keep的范围即可，例如perm_keep[:num_mask_nodes]
idx_keep = idx_keep_dis[perm_keep[:num_mask_nodes]]
idx_mask = idx_mask_dis  

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()



# Cora 0.25
test->train /fixed  remask: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train  31.89%
test->      /random remask: torch.cat((idx_test, idx_val))[:num_mask_nodes]               57.29%

train->     /random remask: idx_train->                                                   55.68%
train->test /fixed  remask: idx_train->torch.cat((idx_test, idx_val))                     58.10%



# Citeseer 0.25
test->train /fixed  remask: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train  49.53
test->      /random remask: torch.cat((idx_test, idx_val))[:num_mask_nodes]               51.84  #这里用mask=0.1，可以说的通，可见范围不变，只是所用的比例不同，要不mask=0.5的话数据差距有点大

train->     /random remask: idx_train->                                                   65.64
train->test /fixed  remask: idx_train->torch.cat((idx_test, idx_val))[:num_mask_nodes]    66.65           

seed 1/2/3


Strategy                        & Attacked-Cora                  & Attacked-Citeseer            & DScore
test->train /fixed  remask      & 31.89%/31.29%/32.39%           & 49.53%/49.29%/49.06%         & 1.0
test->      /random remask      & 51.81%/52.36%/50.15%           & 57.29%/58.95%/59.60%         & 0.1

random      /random remask      & 45.22%/45.32%/44.25%           & 64.45%/64.16%/62.44%         & 0.54
random      /fixed  remask      & 47.89%/46.38%/47.43%           & 64.45%/63.68%/61.14%         & 0.54

train->     /random  remask     & 55.68%/57.24%/56.14%           & 65.64%/64.75%/64.81%         & 0.002
train->test /fixed   remask     & 58.10%/56.34%/57.49%           & 66.65%/66.05%/66.00%         & 0.0

# node center cosine similarity 分析
# 对于Meta_Self cora 0.25  train/val/test 0.1/0.1/0.8
攻击之后的train node  91605
攻击之前的train node  310014

# 实验现象发现 #1-4 四个参数 
让keep的可见范围大一些，但每个epoch取的比例小一点以增加随机性， mask范围小一些，即把目光更聚集在后面的节点上，mask的比例大一些，尽量把这些大的节点全都重建，这样的效果要好很多，比如keep可见前900个，每个epoch取的比例为0.4，mask只取排序2200以后的节点，即最后200个具有最好的sim的节点，每个epoch的重建比例为0.8，这样的训练效果能在0.5719


######### node center cosine similarity masking
idx_keep_dis = g.ndata['csim_sorted'][:900]                          #1
idx_mask_dis = g.ndata['csim_sorted'][2200:]                         #2
perm_keep  = torch.randperm(len(idx_keep_dis), device=x.device)
perm_mask  = torch.randperm(len(idx_mask_dis), device=x.device)

num_dis_keep_nodes = int(len(idx_keep_dis) * 0.4)                    #3
num_dis_mask_nodes = int(len(idx_mask_dis) * 0.8)                    #4
idx_keep   = idx_keep_dis[perm_keep[:num_dis_keep_nodes]]
idx_mask   = idx_mask_dis[perm_mask[:num_dis_mask_nodes]]  

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()
num_mask_nodes = len(mask_nodes)


GraphMAE                                       (很奇怪啊，攻击比例大于0.1反而效果更好了，这方法有问题的感觉)
polblogs meta      DICE				random          heuristic
0.9476±0.0036	   0.9476±0.0036   0.9476±0.0036    0.0   95.70(gcn)   0.9476±0.0036(GraphMAE)
0.8811±0.0275	   0.8592±0.0026   0.9156±0.0049    0.05  78.52(gcn)   0.7843±0.0589(GraphMAE)
0.8142±0.0570	   0.7922±0.0017   0.9095±0.0026    0.1   74.43(gcn)   0.7436±0.0329(GraphMAE)
0.7014±0.0093	   0.7354±0.0023   0.8924±0.0023
0.6777±0.0099	   0.6897±0.0017   0.8865±0.0014
0.6667±0.0067	   0.6414±0.0015   0.8735±0.0020

和鲁棒性方法比较的参数设置 在预训练过程中使用不同的鲁棒方法，但是在下游任务上依然对原始图进行预测 seed 1 
jaccard运行比较快，Guard和SVD超级慢,而且svd太差了，基本就是失效状态

GraphMAE+GATGuard  编码器采用GATGuard 解码器依然沿用GAT
GraphMAE+Jaccard   Cora   threshold 0.06 当不攻击的时候threshold 0.0
                   CoraML 0.25 threshold 0.1   
                          0.15 threshold 0.06  
                          当不攻击的时候threshold 0.04 稍加修剪效果可以85-86以上
GraphMAE+svd       k=50